## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from ns_extension.wrapper import Splatter, SplatterConfig

[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/17/25 21:17:16.630 258700] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [3]:
splatter.mesh()


Available runs:
[0] 2025-07-11_171420


In [4]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"]
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml


[21:17:45] Auto image downscale factor of 2                                                 ]8;id=407783;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=72076;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[21:17:52] use color only optimization with sigmoid activation                                         ]8;id=331606;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=576279;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

In [6]:
splatter.plot_mesh(attribute=similarity)

Number of points: 137342
Number of cells: 253479
Bounds: BoundsTuple(x_min=-5.775000095367432, x_max=0.7600077986717224, y_min=-0.12439944595098495, y_max=7.386317729949951, z_min=-4.992317199707031, z_max=6.735000133514404)


Widget(value='<iframe src="http://localhost:46187/index.html?ui=P_0x7f625cc07c40_1&reconnect=auto" class="pyvi…

In [7]:
import pyvista as pv

# Load the PLY file
mesh = pv.read(splatter.config['mesh_info']['mesh'])

In [8]:
mesh

PolyData (0x7f5f641d1660)
  N Cells:    253479
  N Points:   137342
  N Strips:   0
  X Bounds:   -5.775e+00, 7.600e-01
  Y Bounds:   -1.244e-01, 7.386e+00
  Z Bounds:   -4.992e+00, 6.735e+00
  N Arrays:   1

In [10]:
# Create a plotter and add the mesh
plotter = pv.Plotter()

# if attribute is not None:
#     plotter.add_mesh(mesh, scalars=attribute, rgb=False)
# else:
plotter.add_mesh(mesh, scalars="RGB", rgb=True)

plotter.show_axes()
plotter.show()

Widget(value='<iframe src="http://localhost:46187/index.html?ui=P_0x7f5f748e5e10_3&reconnect=auto" class="pyvi…